# Análise de Consumo de Energia em Escritório – Smart Office Verde

Este notebook faz parte da Global Solution – **Soluções em Energias Renováveis e Sustentáveis**  
Curso: Ciência da Computação – 2º semestre de 2025.

Objetivos principais:

- Carregar e analisar dados de consumo de energia em estações de trabalho.
- Separar consumo em horário de expediente e fora de expediente.
- Identificar consumo potencialmente desperdiçado (sem presença de pessoas).
- Simular o impacto de uma automação IoT que desliga equipamentos quando não há presença.
- Estimar ganhos econômicos e ambientais (R$ e kg de CO₂).


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option("display.max_rows", 10)
pd.set_option("display.max_columns", None)


## 1. Carregando os dados

In [ ]:
# Caminho para o arquivo de dados
csv_path = "../dados/consumo_simulado_escritorio.csv"

dados = pd.read_csv(csv_path)

# Converter data e hora para um único timestamp
dados["datetime"] = pd.to_datetime(dados["data"] + " " + dados["hora"])
dados = dados.sort_values(["datetime", "estacao"]).reset_index(drop=True)

dados.head()


## 2. Visão geral dos dados

In [ ]:
print("Número de linhas:", len(dados))
print("\nColunas:")
print(dados.columns.tolist())

dados.describe(include="all")


## 3. Separando horário de expediente e fora de expediente

In [ ]:
# Definir horário de expediente (ajuste conforme desejado)
inicio_expediente = 8
fim_expediente = 18

dados["hora_int"] = dados["datetime"].dt.hour
dados["em_expediente"] = dados["hora_int"].between(inicio_expediente, fim_expediente)
dados["fora_expediente"] = ~dados["em_expediente"]

# Consumo total (Wh)
consumo_total_wh = dados["consumo_wh"].sum()

consumo_expediente_wh = dados.loc[dados["em_expediente"], "consumo_wh"].sum()
consumo_fora_expediente_wh = dados.loc[dados["fora_expediente"], "consumo_wh"].sum()

print(f"Consumo total (Wh): {consumo_total_wh:,.0f}")
print(f"Consumo em horário de expediente (Wh): {consumo_expediente_wh:,.0f}")
print(f"Consumo fora de expediente (Wh): {consumo_fora_expediente_wh:,.0f}")

print(f"Percentual fora de expediente: {100 * consumo_fora_expediente_wh / consumo_total_wh:.2f}%")


## 4. Identificando consumo potencialmente desperdiçado

In [ ]:
# Consideramos potencial desperdício quando não há pessoas presentes
dados["sem_presenca"] = dados["pessoas_presentes"] == 0

consumo_sem_presenca_wh = dados.loc[dados["sem_presenca"], "consumo_wh"].sum()
print(f"Consumo com 'sem_presenca' (Wh): {consumo_sem_presenca_wh:,.0f}")
print(f"Percentual do total: {100 * consumo_sem_presenca_wh / consumo_total_wh:.2f}%")


## 5. Consumo médio por hora do dia

In [ ]:
consumo_por_hora = dados.groupby("hora_int")["consumo_wh"].mean()

plt.figure()
consumo_por_hora.plot(kind="bar")
plt.xlabel("Hora do dia")
plt.ylabel("Consumo médio por estação (Wh)")
plt.title("Consumo médio por hora do dia (por estação)")
plt.tight_layout()
plt.show()


## 6. Simulação de automação IoT

In [ ]:
# Ideia da simulação:
# - Se não há pessoas presentes na estação naquela hora, assumimos que
#   parte dos equipamentos poderia ser desligada automaticamente.
# - Consideramos que o computador pode ficar em standby em alguns casos,
#   mas monitor, iluminação e ar-condicionado devem ser desligados.

# Fatores de potência aproximados (os mesmos usados na simulação original)
pc_w = 120
monitor_w = 40
light_w = 30
ac_w = 800

def consumo_modelado(row):
    return (
        row["computador"] * pc_w
        + row["monitor"] * monitor_w
        + row["iluminacao"] * light_w
        + row["ar_condicionado"] * ac_w
    )

# Confere se o consumo registrado bate com o modelo aproximado (apenas para referência)
dados["consumo_modelado_wh"] = dados.apply(consumo_modelado, axis=1)

# Agora simulamos a automação IoT:
# - Se sem_presenca == True:
#   - Desligar monitor, iluminação e ar-condicionado
#   - Computador pode permanecer eventualmente em standby (aqui mantemos como está)

def consumo_iot(row):
    if row["sem_presenca"]:
        computador = row["computador"]
        monitor = 0
        iluminacao = 0
        ar_condicionado = 0
    else:
        computador = row["computador"]
        monitor = row["monitor"]
        iluminacao = row["iluminacao"]
        ar_condicionado = row["ar_condicionado"]
    return (
        computador * pc_w
        + monitor * monitor_w
        + iluminacao * light_w
        + ar_condicionado * ac_w
    )

dados["consumo_iot_wh"] = dados.apply(consumo_iot, axis=1)

consumo_total_iot_wh = dados["consumo_iot_wh"].sum()
economia_wh = consumo_total_wh - consumo_total_iot_wh

print(f"Consumo total original (Wh): {consumo_total_wh:,.0f}")
print(f"Consumo total com IoT (Wh): {consumo_total_iot_wh:,.0f}")
print(f"Economia estimada (Wh): {economia_wh:,.0f}")
print(f"Economia percentual: {100 * economia_wh / consumo_total_wh:.2f}%")


## 7. Impacto econômico e ambiental

In [ ]:
# Converter Wh para kWh
consumo_total_kwh = consumo_total_wh / 1000
consumo_total_iot_kwh = consumo_total_iot_wh / 1000
economia_kwh = economia_wh / 1000

# Assumir uma tarifa de energia (R$/kWh) e fator de emissão (kg CO2/kWh)
tarifa_r_kwh = 0.80      # exemplo
fator_co2_kg_kwh = 0.084 # exemplo

custo_original_r = consumo_total_kwh * tarifa_r_kwh
custo_iot_r = consumo_total_iot_kwh * tarifa_r_kwh
economia_r = economia_kwh * tarifa_r_kwh

emissao_original_kg = consumo_total_kwh * fator_co2_kg_kwh
emissao_iot_kg = consumo_total_iot_kwh * fator_co2_kg_kwh
emissao_evitar_kg = economia_kwh * fator_co2_kg_kwh

print(f"Consumo original (kWh): {consumo_total_kwh:,.2f}")
print(f"Consumo com IoT (kWh): {consumo_total_iot_kwh:,.2f}")
print(f"Economia (kWh): {economia_kwh:,.2f}\n")

print(f"Custo original (R$): {custo_original_r:,.2f}")
print(f"Custo com IoT (R$): {custo_iot_r:,.2f}")
print(f"Economia estimada (R$): {economia_r:,.2f}\n")

print(f"Emissão original (kg CO2): {emissao_original_kg:,.2f}")
print(f"Emissão com IoT (kg CO2): {emissao_iot_kg:,.2f}")
print(f"Emissão evitada (kg CO2): {emissao_evitar_kg:,.2f}")


## 8. Comparação visual – cenário original x cenário com IoT

In [ ]:
resumo = pd.DataFrame({
    "cenario": ["Original", "Com IoT"],
    "consumo_kwh": [consumo_total_kwh, consumo_total_iot_kwh],
    "custo_r": [custo_original_r, custo_iot_r],
    "emissao_kg": [emissao_original_kg, emissao_iot_kg],
})

display(resumo)

plt.figure()
plt.bar(resumo["cenario"], resumo["consumo_kwh"])
plt.ylabel("Consumo total (kWh)")
plt.title("Comparação de consumo – Original vs. IoT")
plt.tight_layout()
plt.show()

plt.figure()
plt.bar(resumo["cenario"], resumo["custo_r"])
plt.ylabel("Custo estimado (R$)")
plt.title("Comparação de custo – Original vs. IoT")
plt.tight_layout()
plt.show()

plt.figure()
plt.bar(resumo["cenario"], resumo["emissao_kg"])
plt.ylabel("Emissão estimada (kg CO₂)")
plt.title("Comparação de emissões – Original vs. IoT")
plt.tight_layout()
plt.show()


## 9. Conclusões

- Foi possível identificar uma fração relevante do consumo total associada a períodos sem presença de pessoas,
  caracterizando potencial desperdício.
- A simulação de automação IoT, ainda que simplificada, demonstrou uma economia percentual de energia e,
  consequentemente, de custo e emissões de CO₂.
- A metodologia pode ser facilmente adaptada para dados reais de um escritório, bastando substituir o arquivo CSV
  pelos registros reais de consumo e presença.
- Em um contexto de futuro do trabalho, com modelos híbridos e maior variabilidade de uso dos espaços, soluções
  deste tipo tornam-se ainda mais relevantes para garantir eficiência energética e sustentabilidade.
